In [ ]:
import numpy as np

def er_graph_adj_matrix(N, p):
    A = np.zeros((N, N), dtype=int)

    # fill only the upper triangle
    for i in range(N):
        for j in range(i + 1, N):
            if np.random.rand() < p:
                A[i, j] = 1
                A[j, i] = 1

    return A


In [ ]:
A = er_graph_adj_matrix(50, 0.2)
print(A)

Excercise 3.
Draw resulting graph


In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

def draw_er_graph(N, p):
    A = er_graph_adj_matrix(N, p)
    G = nx.from_numpy_array(A)
    pos = nx.spring_layout(G)
    plt.figure()
    nx.draw(G, pos, with_labels=True)
    plt.show()
    return A, G

A, G = draw_er_graph(50, 0.2)


Excercise 4.
Draw histogram of degree distribution.

In [ ]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

def draw_degree_histogram(A):
    G = nx.from_numpy_array(A)

    degrees = [d for n, d in G.degree()]

    plt.figure()
    plt.hist(degrees, bins=range(0, max(degrees) + 2))
    plt.xlabel("Vertex degree")
    plt.ylabel("Number of vertices")
    plt.title("Degree distribution histogram")
    plt.show()

N = 50
p = 0.2

A = er_graph_adj_matrix(N, p)
draw_degree_histogram(A)


Excercise 5.
What degree of vertex distribution do we expect?


In an Erdős–Rényi graph **G(N, p)**, the degree of each vertex follows a **binomial distribution**:

$$
\deg(v) \sim \text{Binomial}(N-1,\, p)
$$

**Expected degree:**

$$
\mathbb{E}[\deg(v)] = (N-1)p
$$





Excercise 6.
Give the mathematical justification for the Poisson approximation
used.

In an Erdős–Rényi graph \(G(N,p)\), the degree of a fixed vertex has a binomial distribution
$$
\deg(v) \sim \text{Binomial}(N-1, p).
$$

Let
$$
n = N - 1, \quad X \sim \text{Binomial}(n, p).
$$
We choose
$$
p = \frac{\lambda}{n},
$$
so that
$$
\mathbb{E}[X] = np = \lambda
$$
stays fixed as \(n \to \infty\).

The binomial pmf is
$$
\mathbb{P}(X = k)
= \binom{n}{k} p^k (1-p)^{n-k}
= \binom{n}{k} \left(\frac{\lambda}{n}\right)^k
\left(1 - \frac{\lambda}{n}\right)^{n-k}.
$$

We now take the limit as \(n \to \infty\) for fixed \(k\).

First term:
$$
\binom{n}{k} \left(\frac{\lambda}{n}\right)^k
= \frac{n(n-1)\cdots(n-k+1)}{k!} \cdot \frac{\lambda^k}{n^k}
= \frac{\lambda^k}{k!} \prod_{j=0}^{k-1} \left(1 - \frac{j}{n}\right)
\longrightarrow \frac{\lambda^k}{k!}.
$$

Second term:
$$
\left(1 - \frac{\lambda}{n}\right)^{n-k}
= \left(1 - \frac{\lambda}{n}\right)^n
  \left(1 - \frac{\lambda}{n}\right)^{-k}
\longrightarrow e^{-\lambda} \cdot 1
= e^{-\lambda}.
$$

Combining these limits:
$$
\mathbb{P}(X = k)
\longrightarrow e^{-\lambda} \frac{\lambda^k}{k!},
$$
which is exactly the pmf of a Poisson\((\lambda)\) distribution.

Therefore, for large \(N\) and small \(p\) with \((N-1)p = \lambda\) fixed, the degree of a vertex in \(G(N,p)\) is well approximated by a Poisson\((\lambda)\) distribution.


Excercise 7.
Plot both the simulation results and analytically obtained
distributions on one graph. Test appropriate hypotheses.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import binom, poisson, chisquare

def compare_empirical_and_analytical(N, p):
    A = er_graph_adj_matrix(N, p)

    degrees = np.array([sum(row) for row in A])

    k = np.arange(0, N)

    # Observed counts on full support
    counts_full = np.zeros_like(k, dtype=int)
    values, counts = np.unique(degrees, return_counts=True)
    counts_full[values] = counts

    # Empirical probabilities
    prob_emp = counts_full / counts_full.sum()

    # Analytical Binomial distribution
    prob_binom = binom.pmf(k, N - 1, p)

    # Analytical Poisson approximation
    lam = (N - 1) * p
    prob_poisson = poisson.pmf(k, lam)

    plt.figure(figsize=(10, 6))
    plt.bar(k, prob_emp, alpha=0.5, label="Empirical (simulation)")
    plt.plot(k, prob_binom, "o-", label="Binomial")
    plt.plot(k, prob_poisson, "s--", label="Poisson approximation")

    plt.xlabel("Degree k")
    plt.ylabel("Probability")
    plt.title("Degree distribution: empirical vs analytical")
    plt.legend()
    plt.grid(True)
    plt.show()

    # Chi-square tests
    n_vertices = len(degrees)

    # Expected counts for binomial (already sums  - n_vertices)
    exp_binom = prob_binom * n_vertices

    # For Poisson, renormalize because we truncate the tail at k = 0..N-1
    prob_poisson_trunc = prob_poisson / prob_poisson.sum()
    exp_poisson = prob_poisson_trunc * n_vertices

    chi_binom = chisquare(counts_full, f_exp=exp_binom)
    chi_poisson = chisquare(counts_full, f_exp=exp_poisson)

    print("Chi-square test (Binomial fit):", chi_binom)
    print("Chi-square test (Poisson fit):", chi_poisson)


# Run it:
compare_empirical_and_analytical(50, 0.2)


Excercises 8.
Check dependence of the results of the previous excercise for
various values of p and N.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import binom, poisson, chisquare

def fit_degree_distribution_once(N, p):
    A = er_graph_adj_matrix(N, p)

    degrees = np.array([sum(row) for row in A])

    k = np.arange(0, N)

    counts_full = np.zeros_like(k, dtype=int)
    values, counts = np.unique(degrees, return_counts=True)
    counts_full[values] = counts

    n_vertices = len(degrees)

    # Binomial probabilities and expected counts
    prob_binom = binom.pmf(k, N - 1, p)
    exp_binom = prob_binom * n_vertices

    # Poisson probabilities and expected counts (truncated, renormalized)
    lam = (N - 1) * p
    prob_poisson = poisson.pmf(k, lam)
    prob_poisson_trunc = prob_poisson / prob_poisson.sum()
    exp_poisson = prob_poisson_trunc * n_vertices

    # Chi-square tests
    chi_binom = chisquare(counts_full, f_exp=exp_binom)
    chi_poisson = chisquare(counts_full, f_exp=exp_poisson)

    return chi_binom.pvalue, chi_poisson.pvalue


In [ ]:
def explore_N_p_dependence():
    N_list = [50, 100, 200, 500]
    p_list = [0.01, 0.05, 0.1, 0.2]
    n_trials = 20  # number of simulations per (N, p)

    results = []

    for N in N_list:
        for p in p_list:
            binom_pvals = []
            pois_pvals = []
            for _ in range(n_trials):
                pb, pp = fit_degree_distribution_once(N, p)
                binom_pvals.append(pb)
                pois_pvals.append(pp)

            mean_binom = np.mean(binom_pvals)
            mean_pois = np.mean(pois_pvals)

            results.append((N, p, mean_binom, mean_pois))
            print(f"N={N:4d}, p={p:5.3f} -> "
                  f"Binom p≈{mean_binom:.3f}, Poisson p≈{mean_pois:.3f}")

    return results

results = explore_N_p_dependence()


Check the above analytical result by simulation.

In [ ]:
import matplotlib.pyplot as plt

def simulate_clustering(N, p, n_runs=100):
    Cs = []

    for _ in range(n_runs):
        A = er_graph_adj_matrix(N, p)
        G = nx.from_numpy_array(A)

        # clustering coefficient of the whole graph
        C = nx.average_clustering(G)
        Cs.append(C)

    return np.array(Cs)

N = 200
p_values = [0.01, 0.05, 0.1, 0.2]
n_runs = 50

results = []

for p in p_values:
    Cs = simulate_clustering(N, p, n_runs)
    C_mean = Cs.mean()
    results.append((p, C_mean))
    print(f"p = {p:.2f},  simulated <C> ≈ {C_mean:.4f},  analytical = {p:.2f}")

# Plot results
ps = [r[0] for r in results]
C_means = [r[1] for r in results]

plt.figure(figsize=(8,5))
plt.plot(ps, C_means, "o-", label="Simulated ⟨C⟩")
plt.plot(ps, ps, "r--", label="Analytical ⟨C⟩ = p")
plt.xlabel("p")
plt.ylabel("Average clustering coefficient")
plt.title("Clustering coefficient in G(N,p): simulation vs theory")
plt.legend()
plt.grid(True)
plt.show()


P5.3 Generate and draw a graph consisting of 4 community each with
N = 20 nodes and the probability of connection within the
community higher than between them. Draw the result. How it
depends on the parameter values? [2P]

In [ ]:
def community_graph_adj_matrix(num_communities=4, nodes_per_community=20,
                               p_in=0.4, p_out=0.02):
    N = num_communities * nodes_per_community
    A = np.zeros((N, N), dtype=int)

    for i in range(N):
        for j in range(i + 1, N):
            comm_i = i // nodes_per_community
            comm_j = j // nodes_per_community
            # choose probability depending on whether same community
            p = p_in if comm_i == comm_j else p_out
            if np.random.rand() < p:
                A[i, j] = 1
                A[j, i] = 1
    return A

def draw_community_graph(num_communities=4, nodes_per_community=20,
                         p_in=0.4, p_out=0.02):
    A = community_graph_adj_matrix(num_communities, nodes_per_community,
                                   p_in, p_out)
    G = nx.from_numpy_array(A)

    # community labels
    N = num_communities * nodes_per_community
    communities = [i // nodes_per_community for i in range(N)]

    pos = nx.spring_layout(G)

    plt.figure(figsize=(7, 7))
    nx.draw(
        G,
        pos,
        node_color=communities,
        cmap=plt.cm.Set1,
        with_labels=False,
        node_size=80
    )
    plt.title(f"4-community graph, N=20 per community\np_in={p_in}, p_out={p_out}")
    plt.show()

    return A, G

# Example run for the assignment:
A, G = draw_community_graph(
    num_communities=4,
    nodes_per_community=20,
    p_in=0.4,
    p_out=0.02
)


P5.4 Draw a graph of the averaged coefficient of clustering of the WS
network against its parameter p. [1.5P]


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def compute_ws_clustering(N=200, k=6, p_values=None, runs=20):
    if p_values is None:
        p_values = np.linspace(0, 1, 20)

    C_means = []

    for p in p_values:
        Cs = []
        for _ in range(runs):
            G = nx.watts_strogatz_graph(N, k, p)
            C = nx.average_clustering(G)
            Cs.append(C)
        C_means.append(np.mean(Cs))

    return p_values, np.array(C_means)

N = 200
k = 6
runs = 20
p_values = np.linspace(0, 1, 20)

ps, Cs = compute_ws_clustering(N, k, p_values, runs)

# Plot
plt.figure(figsize=(8, 5))
plt.plot(ps, Cs, "o-", label="WS average clustering")
plt.xlabel("Rewiring probability p")
plt.ylabel("Average clustering coefficient")
plt.title("Clustering coefficient of Watts–Strogatz network vs p")
plt.grid(True)
plt.legend()
plt.show()


Implement configuration model and test when the procedure
converge.

In [ ]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import random

def random_edge_swap(G, num_swaps, max_tries_per_swap=10):
    G = G.copy()
    edges = list(G.edges())
    m = len(edges)

    successful_swaps = 0

    while successful_swaps < num_swaps:
        # Try a few times to find a valid swap
        for _ in range(max_tries_per_swap):
            # pick two different edges at random
            (u, v) = edges[random.randrange(m)]
            (x, y) = edges[random.randrange(m)]
            if len({u, v, x, y}) < 4:
                continue  # edges share a node, skip

            # try both possible rewirings
            if random.random() < 0.5:
                a, b = u, x
                c, d = v, y
            else:
                a, b = u, y
                c, d = v, x

            # avoid self-loops
            if a == b or c == d:
                continue

            # avoid creating multi-edges
            if G.has_edge(a, b) or G.has_edge(c, d):
                continue

            # perform the swap
            G.remove_edge(u, v)
            G.remove_edge(x, y)
            G.add_edge(a, b)
            G.add_edge(c, d)

            # update edge list
            edges = list(G.edges())
            m = len(edges)

            successful_swaps += 1
            break
        else:
            break

    return G


In [ ]:
def test_configuration_model_convergence():
    N = 200
    k = 6
    p_ws = 0.1
    G0 = nx.watts_strogatz_graph(N, k, p_ws)

    m = G0.number_of_edges()

    # different levels of randomization: swaps per edge
    swaps_per_edge_list = [0, 0.5, 1, 2, 5, 10, 20]

    clustering_values = []

    for s in swaps_per_edge_list:
        num_swaps = int(s * m)
        G = random_edge_swap(G0, num_swaps)

        C = nx.average_clustering(G)
        clustering_values.append(C)

        print(f"swaps/edge = {s:4.1f}, swaps = {num_swaps:5d}, "
              f"avg clustering = {C:.4f}")

    plt.figure(figsize=(8, 5))
    plt.plot(swaps_per_edge_list, clustering_values, "o-")
    plt.xlabel("Number of swaps per edge")
    plt.ylabel("Average clustering coefficient")
    plt.title("Configuration model convergence (degree-preserving rewiring)")
    plt.grid(True)
    plt.show()

test_configuration_model_convergence()


## Partition Function and Probability Distribution for Networks with a Fixed Number of Edges

### Theory

We consider an ensemble of all simple undirected graphs with:
- a fixed number of nodes \(N\),
- a fixed number of edges \(M\),
- a Hamiltonian \(H(G)\) defined for each graph \(G\).

The **partition function** of this ensemble is:

$$
Z(\beta) = \sum_{G} e^{-\beta H(G)},
$$

where the sum runs over all graphs with exactly \(M\) edges.
This defines the **Boltzmann probability distribution** over graphs:

$$
P(G) = \frac{e^{-\beta H(G)}}{Z(\beta)}.
$$

For small \(N\), we can enumerate all such graphs exactly and compute \(Z\) without approximation.

---

### Code explanation

1. `all_graphs_fixed_edges(N, M)`
   Generates **all simple graphs** with \(N\) nodes and exactly \(M\) edges by taking all \(M\)-element subsets of possible edges.

2. `example_hamiltonian(G)`
   Defines a sample Hamiltonian
   \( H(G) = -J \times (\text{number of triangles}) \).
   You can replace this with any other energy function.

3. `compute_partition_function_and_distribution(...)`
   - enumerates all graphs,
   - computes their energies,
   - evaluates Boltzmann weights \(e^{-\beta H(G)}\),
   - calculates the partition function
     \( Z = \sum_G e^{-\beta H(G)} \),
   - computes the normalized probabilities
     \( P(G) = e^{-\beta H(G)} / Z \).

4. The last cell prints the partition function and, for each graph, its:
   - energy \(H(G)\),
   - probability \(P(G)\).

This provides an exact statistical-mechanical description of the graph ensemble for small \(N\).


In [ ]:
import itertools
import numpy as np
import networkx as nx

def all_graphs_fixed_edges(N, M):
    nodes = list(range(N))
    possible_edges = list(itertools.combinations(nodes, 2))

    if M > len(possible_edges):
        raise ValueError("M is larger than the total possible edges")

    # all combinations of M edges
    for edge_subset in itertools.combinations(possible_edges, M):
        G = nx.Graph()
        G.add_nodes_from(nodes)
        G.add_edges_from(edge_subset)
        return_graph = G
        yield return_graph


def example_hamiltonian(G, J=1.0):
    triangles_per_node = nx.triangles(G)
    num_triangles = sum(triangles_per_node.values()) / 3
    return -J * num_triangles


def compute_partition_function_and_distribution(N, M, beta, H_func):
    energies = []
    graphs = []

    for G in all_graphs_fixed_edges(N, M):
        E = H_func(G)
        energies.append(E)
        graphs.append(G)

    energies = np.array(energies, dtype=float)
    boltz_weights = np.exp(-beta * energies)

    Z = boltz_weights.sum()
    probs = boltz_weights / Z

    return Z, energies, probs, graphs


N = 5
M = 4
beta = 1.0

Z, energies, probs, graphs = compute_partition_function_and_distribution(
    N=N,
    M=M,
    beta=beta,
    H_func=example_hamiltonian
)

print(f"Partition function Z = {Z:.6f}")
print("\nList of all graphs with energies and probabilities:\n")
for i, (E, p) in enumerate(zip(energies, probs)):
    print(f"Graph {i:2d} | H = {E:6.3f} | P(G) = {p:.6f}")
